In [1]:
import sys
import sklearn
assert sklearn.__version__ >= "0.20"

# 공통 모듈 임포트
import numpy as np
import os

# 노트북 실행 결과를 동일하게 유지하기 위해
np.random.seed(42)

# 깔끔한 그래프 출력을 위해
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

In [5]:
#데이터 생성
from sklearn.datasets import make_moons
X,y=make_moons(n_samples=1000,noise=0.4)

In [6]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test =  train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

params={"max_leaf_nodes":[2,3,4,5,6,7,8,9,10],
       "min_samples_split":[2,3,4,5]}
grid_clf=GridSearchCV(DecisionTreeClassifier(random_state=42),params,cv=4)
grid_clf.fit(X_train,y_train)

GridSearchCV(cv=4, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10],
       

In [8]:
grid_clf.best_params_

{'max_leaf_nodes': 4, 'min_samples_split': 2}

In [11]:
grid_clf.best_estimator_

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=4,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='best')

In [10]:
from sklearn.metrics import accuracy_score
y_pred=grid_clf.best_estimator_.predict(X_test)
accuracy_score(y_test, y_pred)

0.855

연습문제 8

In [15]:
from sklearn.model_selection import ShuffleSplit
rs = ShuffleSplit(n_splits=1000, test_size=len(X_train) - 100, random_state=42) #훈련세트의 서브셋이 각각 100개의 샘플을 담아야하니까 test_size가 그 100개를 뺀 나머지
mini_sets=[]

for mini_train_index, mini_test_index in rs.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))

In [19]:
from sklearn.base import clone
forest = [clone(grid_clf.best_estimator_) for _ in range(1000)]

accuracy_scores = []

for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(X_mini_train, y_mini_train)
    
    y_pred = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))

np.mean(accuracy_scores)


0.817115

In [24]:
from scipy.stats import mode
Y_pred = np.empty([1000, len(X_test)], dtype=np.uint8)

for i, tree in enumerate(forest):
    Y_pred[i] = tree.predict(X_test)

y_pred_mode,cnt=mode(Y_pred,axis=0)

In [26]:
accuracy_score(y_test, y_pred_mode.reshape([-1]))

0.85